In [14]:
# -*- coding: utf-8 -*-
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Flatten, Dense, Dropout
from keras.optimizers import SGD
import math

In [15]:
# 이미지 데이터 경로 설정
train_path = r'/content/drive/MyDrive/음식'
valid_path = r'/content/drive/MyDrive/음식'

test_path = r'/content/drive/MyDrive/음식'

In [16]:
# 이미지 크기 설정
img_width, img_height = 224, 224

# 데이터 제너레이터 생성
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# 배치 크기 조정
batch_size = 64

train_generator = train_datagen.flow_from_directory(train_path, target_size=(img_width, img_height), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(valid_path, target_size=(img_width, img_height), batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_path, target_size=(img_width, img_height), batch_size=batch_size, class_mode='categorical')

Found 400 images belonging to 4 classes.
Found 400 images belonging to 4 classes.
Found 400 images belonging to 4 classes.


In [17]:
# VGG16 모델 로딩 및 학습 레이어 제거
input_tensor = Input(shape=(img_width, img_height, 3))
vgg16_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
last_layer = vgg16_model.output
last_layer = Flatten()(last_layer)
last_layer = Dense(256, activation='relu')(last_layer)
last_layer = Dropout(0.5)(last_layer)
output_layer = Dense(train_generator.num_classes, activation='softmax')(last_layer)

model = Model(inputs=vgg16_model.input, outputs=output_layer)

In [18]:
# 미세 조정을 위한 학습 설정
for layer in model.layers[:10]:
    layer.trainable = False

# 학습률 조정
lr = 0.01
sgd = SGD(lr=lr, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [19]:
# 모델 학습 및 평가
steps_per_epoch = math.floor(train_generator.samples / train_generator.batch_size)
validation_steps = math.floor(valid_generator.samples / valid_generator.batch_size)
test_steps = math.ceil(test_generator.samples / test_generator.batch_size)

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=5, validation_data=valid_generator, validation_steps=validation_steps)

scores = model.evaluate_generator(test_generator, steps=test_steps)
print("Test accuracy: %.2f%%" % (scores[1]*100))

<ipython-input-19-19dca02107cf>:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=5, validation_data=valid_generator, validation_steps=validation_steps)


Epoch 1/5
6/6 [==============================] - 650s 114s/step - loss: 2.3461 - accuracy: 0.3036 - val_loss: 1.3868 - val_accuracy: 0.2318
Epoch 2/5
6/6 [==============================] - 661s 117s/step - loss: 1.3877 - accuracy: 0.2786 - val_loss: 1.3865 - val_accuracy: 0.2474
Epoch 3/5
6/6 [==============================] - 642s 113s/step - loss: 1.3868 - accuracy: 0.2411 - val_loss: 1.4451 - val_accuracy: 0.2500
Epoch 4/5
6/6 [==============================] - 695s 124s/step - loss: 1.4355 - accuracy: 0.2344 - val_loss: 1.3868 - val_accuracy: 0.2500
Epoch 5/5
6/6 [==============================] - 644s 114s/step - loss: 1.3880 - accuracy: 0.2440 - val_loss: 1.3867 - val_accuracy: 0.2474


<ipython-input-19-19dca02107cf>:8: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(test_generator, steps=test_steps)


Test accuracy: 25.00%
